In [1]:
import matplotlib.pyplot as plt 
import os
import pandas as pd
import numpy as np 
import scipy.cluster.hierarchy as shc
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import warnings
import re 
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 25)
pd.set_option("display.precision", 5)
import seaborn as sb
sb.set_style('whitegrid')
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
current_dir=os.path.dirname(os.path.abspath("__file__"))
df0=pd.read_excel(os.path.join(current_dir, 'Дымный(добавил откатку).xlsx'), "Рабочий файл")


In [2]:
df=df0

In [3]:
df

,Сценарий,Блок,Передел,Объем,Тип техники,Марка техники+Инв,Марка техники,Инв,Производительность,Смена,Время выполнения,Дата 1,Дата 2
0,Вариант 1,С1-1,разработка торфов,4244.00000,Бульдозер,D-375/688,D-375,688,2500.0,1-2,2,2023-06-01,2023-06-02
1,Вариант 1,С1-1,погрузка торфов,4244.00000,Экскаватор,PC-800 8/913,PC-800 8,913,1767.0,1,2,2023-06-02,2023-06-04
2,Вариант 1,С1-1,Транспортировка торфов,1414.66667,Самосвал,БелАЗ/1082,БелАЗ,1082,589.0,1,2,2023-06-02,2023-06-04
3,Вариант 1,С1-1,Транспортировка торфов,1414.66667,Самосвал,БелАЗ/1150,БелАЗ,1150,589.0,1,2,2023-06-02,2023-06-04
4,Вариант 1,С1-1,Транспортировка торфов,1414.66667,Самосвал,БелАЗ/1155,БелАЗ,1155,589.0,1,2,2023-06-02,2023-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Вариант 1,С1-9 Бедовый,Погрузка песков,7200.00000,Экскаватор,PC-800 8/798,PC-800 8,798,2400.0,1-2,3,2023-08-02,2023-08-04
203,Вариант 1,С1-9 Бедовый,Транспортировка песков,2400.00000,Самосвал,БелАЗ/296,БелАЗ,296,800.0,1-2,3,2023-08-02,2023-08-04
204,Вариант 1,С1-9 Бедовый,Транспортировка песков,2400.00000,Самосвал,БелАЗ/659,БелАЗ,659,800.0,1-2,3,2023-08-02,2023-08-04
205,Вариант 1,С1-9 Бедовый,Транспортировка песков,2400.00000,Самосвал,БелАЗ/660,БелАЗ,660,800.0,1-2,3,2023-08-02,2023-08-04


In [4]:

df=df.dropna()

In [5]:
# Устанавливаем тип данных для столбцов 'Дата 1' и 'Дата 2'
df['Дата 1'] = pd.to_datetime(df['Дата 1'], format='%Y.%m.%d.')
df['Дата 2'] = pd.to_datetime(df['Дата 2'], format='%Y.%m.%d.')

# Создаем пустой датафрейм для результатов
new_df = pd.DataFrame(columns=df.columns)

# Итерация по строкам датафрейма
for _, row in df.iterrows():
    dates = pd.date_range(start=row['Дата 1'], end=row['Дата 2'], freq='D')
    for date in dates:
        new_row = row.copy()
        new_row['Дата'] = date.strftime('%Y.%m.%d.')
        new_df = new_df.append(new_row, ignore_index=True)



C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(

In [6]:
# Преобразование столбца 'Дата' в тип данных datetime
new_df['Дата'] = pd.to_datetime(new_df['Дата'])

In [7]:
# Извлечение названия месяца в новый столбец 'Месяц'
#new_df['Месяц'] = new_df['Дата'].dt.strftime('%B')


In [8]:
'''# Добавляем столбец "Уменьшающийся объем"
new_df['Уменьшающийся объем'] = new_df['Объем']
# Условная индексация для уменьшающегося объема
mask = (new_df['Передел'] == new_df['Передел'].shift(-1)) & \
       (new_df['Марка техники'] == new_df['Марка техники'].shift(-1)) & \
       (new_df['Блок'] == new_df['Блок'].shift(-1))
new_df.loc[mask, 'Уменьшающийся объем'] -= new_df['Производительность'].shift(-1)
'''

'# Добавляем столбец "Уменьшающийся объем"\nnew_df[\'Уменьшающийся объем\'] = new_df[\'Объем\']\n# Условная индексация для уменьшающегося объема\nmask = (new_df[\'Передел\'] == new_df[\'Передел\'].shift(-1)) &        (new_df[\'Марка техники\'] == new_df[\'Марка техники\'].shift(-1)) &        (new_df[\'Блок\'] == new_df[\'Блок\'].shift(-1))\nnew_df.loc[mask, \'Уменьшающийся объем\'] -= new_df[\'Производительность\'].shift(-1)\n'

In [9]:
for i, row in new_df.iterrows():
    
    if i>=1:     
        if  ((new_df.loc[i,"Передел"]==new_df.loc[i-1,"Передел"])& \
            (new_df.loc[i,"Марка техники"]==new_df.loc[i-1,"Марка техники"])& \
            (new_df.loc[i,"Блок"]==new_df.loc[i-1,"Блок"])& \
            (new_df.loc[i,"Инв"]==new_df.loc[i-1,"Инв"])& \
             (new_df.loc[i,"Дата 1"]==new_df.loc[i-1,"Дата 1"])
            ):
           # Moto4_pokaz.loc[i,"счетчик"]=Moto4_pokaz.loc[i-1,"счетчик"]+1
            new_df.loc[i,"Уменьшающийся объем"]=new_df.loc[i-1,"Уменьшающийся объем"]-min(new_df.loc[i,"Производительность"],new_df.loc[i,"Уменьшающийся объем"])
           
        
    else:
          
            new_df['Уменьшающийся объем'] = new_df['Объем']
          
    

In [10]:
i=0
#=ЕСЛИ(Уменьшающийся объем<0;P586;ЕСЛИ(Уменьшающийся объем=0;P586;ЕСЛИ(И(D587=D586;I587=I586;C587=C586;L587=L586);МИН(Уменьшающийся объем586-Уменьшающийся объем587;Уменьшающийся объем587);МИН(Уменьшающийся объем587;J587))))
for i, row in new_df.iterrows():
    if i >= 1:
        if new_df.loc[i, "Уменьшающийся объем"] <= 0:
            new_df.loc[i, "Выполненный объем"] = new_df.loc[i-1, "Выполненный объем"]
        else:
            if (new_df.loc[i, "Передел"] == new_df.loc[i-1, "Передел"]) and \
               (new_df.loc[i, "Марка техники"] == new_df.loc[i-1, "Марка техники"]) and \
               (new_df.loc[i, "Блок"] == new_df.loc[i-1, "Блок"]) and \
               (new_df.loc[i, "Инв"] == new_df.loc[i-1, "Инв"]) and \
               (new_df.loc[i, "Дата 1"] == new_df.loc[i-1, "Дата 1"]):
                    new_df.loc[i, "Выполненный объем"] = min((new_df.loc[i-1, "Уменьшающийся объем"]-new_df.loc[i, "Уменьшающийся объем"]), new_df.loc[i, "Уменьшающийся объем"])
            else:
                new_df.loc[i, "Выполненный объем"] = min(new_df.loc[i, "Уменьшающийся объем"], new_df.loc[i, 'Производительность'])
    if i == 0:
         new_df.loc[i, "Выполненный объем"]=min(new_df.loc[i, "Уменьшающийся объем"], new_df.loc[i, 'Производительность'])


In [11]:
new_df['Название месяца'] = new_df['Дата'].dt.strftime('%B').str.capitalize()

In [12]:
new_df['Месяц']=new_df['Дата'].dt.month

In [14]:
new_df

,Сценарий,Блок,Передел,Объем,Тип техники,Марка техники+Инв,Марка техники,Инв,Производительность,Смена,Время выполнения,Дата 1,Дата 2,Дата,Уменьшающийся объем,Выполненный объем,Название месяца,Месяц
0,Вариант 1,С1-1,разработка торфов,"4,244",Бульдозер,D-375/688,D-375,688,"2,500",1-2,2,2023-06-01,2023-06-02,2023-06-01,"4,244","2,500",June,6
1,Вариант 1,С1-1,разработка торфов,"4,244",Бульдозер,D-375/688,D-375,688,"2,500",1-2,2,2023-06-01,2023-06-02,2023-06-02,"1,744","1,744",June,6
2,Вариант 1,С1-1,погрузка торфов,"4,244",Экскаватор,PC-800 8/913,PC-800 8,913,"1,767",1,2,2023-06-02,2023-06-04,2023-06-02,"4,244","1,767",June,6
3,Вариант 1,С1-1,погрузка торфов,"4,244",Экскаватор,PC-800 8/913,PC-800 8,913,"1,767",1,2,2023-06-02,2023-06-04,2023-06-03,"2,477","1,767",June,6
4,Вариант 1,С1-1,погрузка торфов,"4,244",Экскаватор,PC-800 8/913,PC-800 8,913,"1,767",1,2,2023-06-02,2023-06-04,2023-06-04,710,710,June,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,Вариант 1,С1-9 Бедовый,Транспортировка песков,"2,400",Самосвал,БелАЗ/660,БелАЗ,660,800,1-2,3,2023-08-02,2023-08-04,2023-08-03,"1,600",800,August,8
733,Вариант 1,С1-9 Бедовый,Транспортировка песков,"2,400",Самосвал,БелАЗ/660,БелАЗ,660,800,1-2,3,2023-08-02,2023-08-04,2023-08-04,800,800,August,8
734,Вариант 1,С1-9 Бедовый,Промывка песков,"7,200",Промприбор,ГИТ-62 МЕ/668,ГИТ-62 МЕ,668,"2,400",1-2,3,2023-08-02,2023-08-04,2023-08-02,"7,200","2,400",August,8
735,Вариант 1,С1-9 Бедовый,Промывка песков,"7,200",Промприбор,ГИТ-62 МЕ/668,ГИТ-62 МЕ,668,"2,400",1-2,3,2023-08-02,2023-08-04,2023-08-03,"4,800","2,400",August,8


In [16]:
new_df['Название месяца'].unique()

array(['June', 'July', 'May', 'August'], dtype=object)

In [18]:
new_df['Название месяца'].replace({'June': 'Июнь', 'July': 'Июль', 'May': 'Май', 'August': 'Август', 'September': 'Сентябрь'})

0        Июнь
1        Июнь
2        Июнь
3        Июнь
4        Июнь
        ...  
732    Август
733    Август
734    Август
735    Август
736    Август
Name: Название месяца, Length: 737, dtype: object

In [19]:
pivot_table = pd.pivot_table(new_df,values=[ 'Производительность'], 
                             index=['Дата'], 
                             columns=['Марка техники+Инв'], 
                             aggfunc={ 'Производительность': 'sum'})


pivot_table = pivot_table.round(2)
pivot_table = pivot_table.fillna(0)

# установка формата вывода
pd.options.display.float_format = '{:,.0f}'.format

# применение формата для всех чисел в таблице
pivot_table = pivot_table.applymap(lambda x: '{:,.0f}'.format(x) if isinstance(x, (int, float)) else x)

# print the updated pivot table
pivot_table


Производительность                                      \
Марка техники+Инв          D-375/677 D-375/688 PC-800 8/798 PC-800 8/913   
Дата                                                                       
2023-05-31                     3,400         0            0        3,534   
2023-06-01                     2,500     2,500        3,534        3,534   
2023-06-02                     2,500     2,500        3,534        3,534   
2023-06-03                     2,500     2,500        3,534        3,534   
2023-06-04                     2,500     2,500        3,535        3,534   
...                              ...       ...          ...          ...   
2023-07-31                         0         0        2,400        3,534   
2023-08-01                         0         0        2,400        3,534   
2023-08-02                     2,500         0        2,400            0   
2023-08-03                     2,500         0        2,400            0   
2023-08-04                     2,500         0        2,400            0   

                                                                        \
Марка техники+Инв БелАЗ/1082 БелАЗ/1150 БелАЗ/1155 БелАЗ/296 БелАЗ/659   
Дата                                                                     
2023-05-31             1,178      1,178      1,178         0         0   
2023-06-01             1,178      1,178      1,178     1,178     1,178   
2023-06-02             1,178      1,178      1,178     1,178     1,178   
2023-06-03             1,178      1,178      1,178     1,178     1,178   
2023-06-04             1,178      1,178      1,178     1,178     1,178   
...                      ...        ...        ...       ...       ...   
2023-07-31             1,178      1,178      1,178       800       800   
2023-08-01             1,178      1,178      1,178       800       800   
2023-08-02                 0          0          0       800       800   
2023-08-03                 0          0          0       800       800   
2023-08-04                 0          0          0       800       800   

                                                        
Марка техники+Инв БелАЗ/660 ГИТ-62 МЕ/668 СБШ-250/1103  
Дата                                                    
2023-05-31                0             0            0  
2023-06-01            1,178             0            0  
2023-06-02            1,178             0            0  
2023-06-03            1,178             0          500  
2023-06-04            1,178             0          500  
...                     ...           ...          ...  
2023-07-31              800         2,400            0  
2023-08-01              800         2,400            0  
2023-08-02              800         2,400            0  
2023-08-03              800         2,400            0  
2023-08-04              800         2,400            0  

[66 rows x 12 columns]

In [20]:
new_df.to_excel(os.path.join(current_dir, 'Дымный_календарь_тест.xlsx'), "Дымный")